# Build Sentinel-2 Data Cubes

This notebook demonstrates how to construct 3D data cubes from multiple Sentinel-2 tiles using [**xcube-eopf**](https://github.com/EOPF-Sample-Service/xcube-eopf). The following steps are performed by **xcube-eopf** to generate the data cubes:

- **Query tiles**: Identify relevant Sentinel-2 tiles based on a spatiotemporal search using the [EOPF Zarr Sample Service STAC API](https://stac.core.eopf.eodc.eu).
- **Group by tile and date**: Organize STAC items by MGRS tile ID and solar day.
- **Spatial mosaicking**: Merge adjacent tiles acquired on the same day into seamless scenes.
- **Time stacking**: Combine daily mosaics along the time dimension to form 3D data cubes for each data variable (e.g., spectral bands).


In [1]:
from xcube.core.store import new_data_store
from xcube_eopf.utils import reproject_bbox
import xarray as xr

In [2]:
xr.set_options(display_expand_attrs=False)

To initialize an `eopf-zarr` [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework), execute the cell below:

In [3]:
store = new_data_store("eopf-zarr")

The data IDs point to [STAC collections](https://stac.browser.user.eopf.eodc.eu). So far only `'sentinel-2-l1c'` and `'sentinel-2-l2a'` is supported.

In [4]:
data_ids = store.list_data_ids()
data_ids

['sentinel-2-l1c', 'sentinel-2-l2a']

Below, the parameters for the `open_data` method can be viewed for each supported data product. 

In [5]:
open_params = store.get_open_data_params_schema()
open_params

This function also shows opening parameters for a specific `data_id`, as shown below. 

In [6]:
open_params = store.get_open_data_params_schema(data_id="sentinel-2-l2a")
open_params

We now want to generate a data cube from the Sentinel-2 L2A product. We therefore assign `data_id` to `"sentinel-2-l2a"`. We set the bounding box to cover the greater Hamburg area and the time range to the first half of Mai 2025. We first want to get the datacube in the native UTM projection. 

In [7]:
bbox = [9.7, 53.4, 10.3, 57.3]
crs_target = "EPSG:32632"
bbox_utm = reproject_bbox(bbox, "EPSG:4326", crs_target)

In [9]:
%%time
ds = store.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox_utm,
    time_range=["2025-05-01", "2025-05-15"],
    spatial_res=10,
    crs=crs_target,
    variables=["b02", "b03", "b04", "scl"],
)
ds

ClientResponseError: 504, message='Gateway Time-out', url='https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:202505-s02msil2a/01/products/cpm_v256/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558.zarr/measurements/reflectance/r10m/y/0'

In [10]:
%%time
ds.b04.isel(time=0)[::10, ::10].plot(vmin=0, vmax=0.2)

NameError: name 'ds' is not defined

Now wecan request the same data cube but in geographic projection (`"EPSG:4326"`). xcube-eopf can reproject the datacube to any projection requested by the user. 

In [11]:
%%time
ds = store.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox,
    time_range=["2025-05-01", "2025-05-15"],
    spatial_res=10 / 111320, # meter in degree,
    crs="EPSG:4326",
    variables=["b02", "b03", "b04", "scl"],
)
ds

ClientResponseError: 504, message='Gateway Time-out', url='https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:202505-s02msil2a/01/products/cpm_v256/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558.zarr/measurements/reflectance/r10m/y/0'

In [12]:
%%time
ds.b04.isel(time=0)[::10, ::10].plot(vmin=0, vmax=0.2)

NameError: name 'ds' is not defined

We now want to generate a similar data cube from the Sentinel-2 L1C product. We therefore assign `data_id` to `"sentinel-2-l1c"`.

In [13]:
# "sentinel-2-l1c" produces 504 error; see issue https://github.com/EOPF-Sample-Service/PMP/issues/207
%%time
ds = store.open_data(
    data_id="sentinel-2-l1c",
    bbox=bbox_utm,
    time_range=["2025-05-01", "2025-05-15"],
    spatial_res=10,
    crs=crs_target,
    variables=["b02", "b03", "b04", "scl"],
)
ds

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time
ds.b04.isel(time=0)[::10, ::10].plot(vmin=0, vmax=0.2)